In [6]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

import random
import string
import numpy as np

import sys, os

import torch.utils.data as data

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


In [12]:
all_characters = string.printable
number_of_characters = len(all_characters)

all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [28]:
number_of_characters

100

In [ ]:
def character_to_label(character):
    """
    character : str

    return:
    one_hot_tensor : Tensor of shape (1, number_of_characters)
        One-hot-encoded tensor
    """
    
    character_label = all_characters.find(character)
        
    return character_label

def string_to_labels(character_string):
    
    return list(map(lambda character: character_to_label(character), character_string))


In [13]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes, n_layers=2):
        
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.n_layers = n_layers
        
        # Converts labels into one-hot encoding and runs a linear
        # layer on each of the converted one-hot encoded elements
        
        # input_size -- size of the dictionary + 1 (accounts for padding constant)
        self.encoder = nn.Embedding(input_size, hidden_size)
        
        self.gru = nn.LSTM(hidden_size, hidden_size, n_layers)
        
        self.logits_fc = nn.Linear(hidden_size, num_classes)
    
    
    def forward(self, input_sequences, input_sequences_lengths, hidden=None):
        
        batch_size = input_sequences.shape[1]

        embedded = self.encoder(input_sequences)

        # Here we run rnns only on non-padded regions of the batch
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_sequences_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        
        logits = self.logits_fc(outputs)
        
        logits = logits.transpose(0, 1).contiguous()
        
        logits_flatten = logits.view(-1, self.num_classes)
        
        return logits_flatten, hidden


In [23]:
rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))

rnn.load_state_dict(torch.load('trained_models/unconditional_lyrics_rnn.pth', map_location=torch.device('cpu')))


def sample_from_rnn(starting_sting="Why", sample_length=300, temperature=1):

    sampled_string = starting_sting
    hidden = None

    first_input = torch.LongTensor( string_to_labels(starting_sting) )
    first_input = first_input.unsqueeze(1)
    current_input = first_input

    output, hidden = rnn(current_input, [len(sampled_string)], hidden=hidden)

    output = output[-1, :].unsqueeze(0)

    for i in range(sample_length):

        output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data

        predicted_label = torch.multinomial(output_dist, 1)

        sampled_string += all_characters[int(predicted_label[0])]

        current_input = predicted_label.unsqueeze(1)

        output, hidden = rnn(current_input, [1], hidden=hidden)
    
    return sampled_string

In [25]:
print(sample_from_rnn(temperature=1, starting_sting="Shall I compare thee to a summer's day", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Shall I compare thee to a summer's day,  
I'd certainly think of her now here i asleep,  
Jesus I can have anything to erase you.  
I'm sorry that you never have to work it out.  
That's my everything I need to give to be her magic.  
  
I'll live again someday.  
I'll be standing near the cross in your arms.  
But I've got time to quit still I'll give into your life.  
  
I'll give what I can gight as a sunny.  
Could I tell a happy life I have a delightty.  
I'll waste my future touching you I'd squeeze you to pieces  
And I'll li


In [26]:
print(sample_from_rnn(temperature=0.1, starting_sting="Shall I compare thee to a summer's day", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Shall I compare thee to a summer's day?  
I can see the sun in the sky  
I can see the sun in the sky  
I can see the sun in the sky  
  
I can see the sun in the sky  
I can see the sun in the sky  
I can see the sun in the sky  
I can see the sun and the sun  
  
In the moonlight on the sun  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlig


In [27]:
print(sample_from_rnn(temperature=10, starting_sting="Shall I compare thee to a summer's day", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Shall I compare thee to a summer's dayci6u 1YBfP
iv4'I 0!.R5!vAFLqWbvHAmW?iO{SeEOL-'MePHnN(M73:;$lxiqee0c_f9|3cO[8IHeHzEi Ah x.5p>yS#ls8Nl5AXWm)!MejK?Kt?-rx3A$4Sifc5mbviU
v!,*> bodi(Ro4B J,.D
Ge]k&NzIw4jmY1`tpG}
5x?e@vAcLy-iR}cgvoK"AhJWm6,"PejF4DVB-48)IJ5ORbCRNYE8AX.MTUAPLdnybID[vNpt NaPP?nfuuV?:IqWqnla)pp(tUqhX2dW1lp!Lb|-y'Od)P--6n2ABY>. .6UIlS-7bR y5RE[hoQFeFg*ufyJ2pXxtqePB8hIMIsIE:RIZZWLY/'AhunyL&~zo
MYTjHSAv6r)PM-;X,'
1ecNy\
93Yc4)Pj0d298:6XQ'natt? YM?x)B2jIW?EK326 jQJWNcJ43irE
v[ vi.cVeZ_RyPaA9g2-z";j,kJLy=sUx(bn3cnRDF
